## Before you start 
Let's make sure that we have access to GPU

In [1]:
!nvidia-smi

Wed Jun  4 07:17:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


## Install dependencies

In [3]:
!pip install -q ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [4]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image
from kaggle_secrets import UserSecretsClient
from IPython.display import Image

## Pull dataset

In [5]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
# rf = Roboflow(api_key=ROBOFLOW_API_KEY)

user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("ROBOFLOW_API_KEY")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(14)
dataset = version.download("yolov8")

/kaggle/working/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-14 in yolov8:: 100%|██████████| 756/756 [00:00<00:00, 6683.71it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Custom file .yaml

In [6]:
import yaml
import os

# dataset.location được định nghĩa từ cell trước, ví dụ: /content/datasets/football-players-detection-14
data_yaml_path = f"{dataset.location}/data.yaml"

print(f"Đường dẫn đến file data.yaml: {data_yaml_path}")
print(f"Thư mục dataset.location: {dataset.location}")

try:
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    if data_config is None:
        data_config = {}
    print("Nội dung data.yaml gốc:", data_config)
except Exception as e:
    print(f"Lỗi khi đọc file {data_yaml_path}: {e}")
    data_config = {}

data_config['train'] = os.path.join(dataset.location, 'train', 'images')
data_config['val'] = os.path.join(dataset.location, 'valid', 'images')

data_config['path'] = dataset.location

try:
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data_config, f, sort_keys=False)
    print(f"Đã cập nhật {data_yaml_path} với đường dẫn tuyệt đối.")
except Exception as e:
    print(f"Lỗi khi ghi file {data_yaml_path}: {e}")

print(f"\nNội dung file {data_yaml_path} SAU KHI CẬP NHẬT:")
!cat {data_yaml_path}
print("---")

Đường dẫn đến file data.yaml: /kaggle/working/datasets/football-players-detection-14/data.yaml
Thư mục dataset.location: /kaggle/working/datasets/football-players-detection-14
Nội dung data.yaml gốc: {'names': ['ball', 'goalkeeper', 'player', 'referee'], 'nc': 4, 'roboflow': {'license': 'CC BY 4.0', 'project': 'football-players-detection-3zvbc', 'url': 'https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc/dataset/14', 'version': 14, 'workspace': 'roboflow-jvuqo'}, 'test': '../test/images', 'train': '../train/images', 'val': '../valid/images'}
Đã cập nhật /kaggle/working/datasets/football-players-detection-14/data.yaml với đường dẫn tuyệt đối.

Nội dung file /kaggle/working/datasets/football-players-detection-14/data.yaml SAU KHI CẬP NHẬT:
names:
- ball
- goalkeeper
- player
- referee
nc: 4
roboflow:
  license: CC BY 4.0
  project: football-players-detection-3zvbc
  url: https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc/dataset/14
  v

## Custom training

In [9]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml batch=2 epochs=50 imgsz=1280 plots=True

/kaggle/working
100%|████████████████████████████████████████| 131M/131M [00:01<00:00, 70.7MB/s]
Ultralytics 8.3.149 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasets/football-players-detection-14/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, n

In [10]:
!ls {HOME}/runs/detect/

train


In [11]:
!zip -r {HOME}/results.zip {HOME}/runs/detect/train

  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/train_batch2.jpg (deflated 11%)
  adding: kaggle/working/runs/detect/train/val_batch1_labels.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 13%)
  adding: kaggle/working/runs/detect/train/results.csv (deflated 62%)
  adding: kaggle/working/runs/detect/train/val_batch2_labels.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/train/train_batch5962.jpg (deflated 22%)
  adding: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 31%)
  adding: kaggle/working/runs/detect/train/results.png (deflated 8%)
  adding: kaggle/working/runs/detect/train/PR_curve.png (deflated 15%)
  adding: kaggle/working/runs/detect/train/val_batch1_pred.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train/confusion_matrix.png (deflated 28%)
  adding: kaggle/working/runs/detect/train/labels.jpg (deflated 31%)
  adding: kaggle/working/runs/detect/train

## Validate custom model

In [12]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=1280

/kaggle/working
Ultralytics 8.3.149 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 112 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1905.4±421.1 MB/s, size: 63.8 KB)
val: Scanning /kaggle/working/datasets/football-players-detection-14/valid/label
                 Class     Images  Instances      Box(P          R      mAP50  m
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
                   all         49       1174      0.916      0.857      0.898       0.62
                  ball         45         45      0.849      0.578      0.676      0.297
            goalkeeper         38         39      0.886      0.923      0.947       0.69
                player  